In [1]:
import numpy as np
import pickle
import os

CIFAR_DIR = '..\\cifar-10-batches-py'

def load_data(filename):
    with open(os.path.join(CIFAR_DIR, filename), 'rb') as f:
        data = pickle.load(f, encoding='bytes')
        return data[b'data'], data[b'labels']
    
class Cifar_data:
    def __init__(self, filenames, need_shuffle):
        all_data = []
        all_labels = []

        for filename in filenames:
            data, labels = load_data(filename)
            for d, l in zip(data, labels):
                if l in [0, 1]:
                    all_data.append(d)
                    all_labels.append(l)
    
        self._data = np.vstack(all_data)                # 0.4865
        self._data = self._data / 127.5 - 1              
        self._labels = np.hstack(all_labels)
        print(self._data.shape)

        self._num_examples = self._data.shape[0]
        self._need_shuffle = need_shuffle
        self._indicator = 0
        if self._need_shuffle:
            self._shuffle_data()
    
    def _shuffle_data(self):
        p = np.random.permutation(self._num_examples)
        self._data = self._data[p]
        self._labels = self._labels[p]

    def next_batch(self, batch_size):
        """return batch_size examples as a batch"""
        end_indicator = self._indicator + batch_size
        if end_indicator > self._num_examples:
            if self._need_shuffle:          
                self._shuffle_data()
                self._indicator = 0
                end_indicator = batch_size
            else:
                raise Exception("have no more examples")
        if end_indicator > self._num_examples:
            raise Exception("batch size is lager than all example")
        batch_data = self._data[self._indicator: end_indicator]
        batch_labels = self._labels[self._indicator: end_indicator]
        self._indicator = end_indicator
        return batch_data, batch_labels

train_filenames = [os.path.join(CIFAR_DIR, "data_batch_%d" % i) for i in range(1, 6)]
test_filename = [os.path.join(CIFAR_DIR, "test_batch")]

train_data = Cifar_data(train_filenames, True)
test_data = Cifar_data(test_filename, False)

(10000, 3072)
(2000, 3072)


In [2]:
import tensorflow as tf

x = tf.placeholder(tf.float32, [None, 3072])
y = tf.placeholder(tf.int64, [None])

w = tf.get_variable('w', [x.get_shape()[-1], 1], initializer=tf.random_normal_initializer(0, 1))
b = tf.get_variable('b', [1], initializer=tf.constant_initializer(0.0))          # 错误 random_normal_initializer

y_ = tf.matmul(x, w) + b
p_y_ = tf.sigmoid(y_)            # ？？

y_reshape = tf.reshape(y, (-1, 1))
y_reshape_float = tf.cast(y_reshape, tf.float32)
loss = tf.reduce_mean(tf.square(y_reshape_float - p_y_))

predict = p_y_ > 0.5
correct_predict = tf.equal(tf.cast(predict, tf.int64), y_reshape)
accuracy = tf.reduce_mean(tf.cast(correct_predict, tf.float64))

with tf.name_scope('train_scope'):
    train_op = tf.train.AdamOptimizer(1e-3).minimize(loss)

In [3]:
loss_summary = tf.summary.scalar('loss', loss)
accuracy_summary = tf.summary.scalar('accuracy', accuracy)

merged_summary = tf.summary.merge_all()
merged_summary_test = tf.summary.merge([loss_summary, accuracy_summary])

LOG_DIR = '.'
run_label = 'run_nn_tensorboard'
run_dir = os.path.join(LOG_DIR, run_label)

if not os.path.exists(run_dir):
    os.mkdir(run_dir)

train_log_dir = os.path.join(run_dir, 'train')
test_log_dir = os.path.join(run_dir, 'test')
if not os.path.exists(train_log_dir):
    os.mkdir(train_log_dir)
if not os.path.exists(test_log_dir):
    os.mkdir(test_log_dir)

In [4]:
init = tf.global_variables_initializer()

batch_size = 20
train_steps = 30000
test_steps = 100
output_summary_every_steps = 100

with tf.Session() as sess:
    sess.run(init)

    train_writer = tf.summary.FileWriter(train_log_dir)
    test_writer = tf.summary.FileWriter(test_log_dir)

    fixed_test_batch_data, fixed_test_batch_labels = test_data.next_batch(batch_size)
    for i in range(train_steps):
        batch_data, batch_labels = train_data.next_batch(batch_size)

        eval_ops = [loss, accuracy, train_op]
        should_output_summary = ((i + 1) % output_summary_every_steps == 0)
        if should_output_summary:
            eval_ops.append(merged_summary)

        eval_ops_results = sess.run([loss, accuracy, train_op], feed_dict = {x: batch_data, y: batch_labels})

        if should_output_summary:
            train_summary_str = eval_ops_results[-1]
            train_writer.add_summary(train_summary_str, i+1)
            test_summary_str = sess.run([merged_summary_test], feed_dict={x: fixed_test_batch_data, y: fixed_test_batch_labels})[0]
            test_writer.add_summary(test_summary_str, i+1)

        loss_val, acc_val = eval_ops_results[0:2]

        if (i+1) % 500 == 0:
            print('[Train] Step：%d, loss: %4.5f, acc: %4.5f' % (i + 1, loss_val, acc_val))
        if (i+1) % 5000 == 0:
            test_data = Cifar_data(test_filename, False)
            all_test_acc_val = []
            for j in range(test_steps):
                test_batch_data, test_batch_labels = test_data.next_batch(batch_size)
                test_acc_val = sess.run([accuracy], feed_dict={x:test_batch_data,y:test_batch_labels})
                all_test_acc_val.append(test_acc_val)
            test_acc = np.mean(all_test_acc_val)
            print('[Test] Step: %d, acc: %4.5f' % (i+1, test_acc))

[Train] Step：500, loss: 0.21369, acc: 0.75000
[Train] Step：1000, loss: 0.24979, acc: 0.75000
[Train] Step：1500, loss: 0.15247, acc: 0.85000
[Train] Step：2000, loss: 0.33463, acc: 0.65000
[Train] Step：2500, loss: 0.34952, acc: 0.65000
[Train] Step：3000, loss: 0.22325, acc: 0.75000
[Train] Step：3500, loss: 0.29974, acc: 0.70000
[Train] Step：4000, loss: 0.05043, acc: 0.95000
[Train] Step：4500, loss: 0.34859, acc: 0.65000
[Train] Step：5000, loss: 0.12762, acc: 0.85000
(2000, 3072)
[Test] Step: 5000, acc: 0.81300
[Train] Step：5500, loss: 0.19653, acc: 0.80000
[Train] Step：6000, loss: 0.21092, acc: 0.75000
[Train] Step：6500, loss: 0.10000, acc: 0.90000
[Train] Step：7000, loss: 0.15000, acc: 0.85000
[Train] Step：7500, loss: 0.05000, acc: 0.95000
[Train] Step：8000, loss: 0.15000, acc: 0.85000
[Train] Step：8500, loss: 0.25022, acc: 0.75000
[Train] Step：9000, loss: 0.15544, acc: 0.85000
[Train] Step：9500, loss: 0.20001, acc: 0.80000
[Train] Step：10000, loss: 0.25006, acc: 0.75000
(2000, 3072)
[T